# Image MEA
1. Setup a project
2. Import labels with MAL
3. Correct the labels in the editor
4. Upload labels and predictions for MEA
------ 

## Environment Setup
* Install dependencies

In [ ]:
!pip install labelbox==2.7b1 \
             requests \
             ndjson \
             scikit-image \
             PILLOW \
             tensorflow \
             opencv-python

In [2]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python && git checkout mea-dev
    !mv labelbox-python/examples/model_assisted_labeling/*.py .

* Import libraries

In [3]:
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend, MALPredictionImport
from image_model import predict, class_mappings, load_model
from io import BytesIO
from getpass import getpass
import uuid
import numpy as np
from PIL import Image
import requests
from collections import defaultdict
import ndjson
import os
from ndjson_utils import (
    create_boxes_ndjson, 
    create_polygon_ndjson, 
    create_mask_ndjson, 
    create_point_ndjson
)
from labelbox.data.metrics.iou import data_row_miou
from labelbox.data.serialization import NDJsonConverter
from labelbox.data.annotation_types import ScalarMetric, LabelList, Label, RasterData, Mask, Point, Rectangle, ObjectAnnotation

* Configure client

In [4]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [5]:
ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=API_KEY, endpoint = ENDPOINT)

In [6]:
load_model()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://cloud-tpu-checkpoints/mask-rcnn/1555659850/variables/variables


## Make Inferences

In [7]:
def get_annotations(boxes, classes, seg_masks):
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        name = class_mappings[class_idx]
        value = None
        classifications = []
        if name in ['person', 'handbag']:
            value = Rectangle(
                start = Point(x = box[1], y = box[0]), end = Point(x = box[3], y = box[2])
            )
        elif name == 'car':
            value = Mask(mask = RasterData.from_2D_arr(arr = seg), color = (1,1,1))            
        if value is not None:
            annotations.append(
                ObjectAnnotation(
                    name = name,
                    value = value
                )
            ) 
    return annotations

### We can just start creating predictions whether or not we have a 
image_paths = ['/Users/matthewsokoloff/Downloads/kitano_st.jpeg']
labellist = LabelList([])

for image_url in image_paths:
    image_data = RasterData(file_path = image_url)
    height, width = image_data.data.shape[:2]
    prediction = predict(np.array([image_data.im_bytes]), min_score=0.5, height=height, width = width)
    annotations = get_annotations(prediction['boxes'], prediction['class_indices'], prediction['seg_masks'])
    labellist.append(Label(
        data = image_data,
        annotations = annotations
    ))
    

## Project Setup

* Define ontology

In [8]:
# Lets setup a project to label
# Note see Ontology, Project, and Project_setup notebooks for more information on this section.
project = client.create_project(name="subclass_mal_project")
dataset = client.create_dataset(name="subclass_mal_dataset")
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == 'editor'))
# Use the label collection to build the ontology
project.setup(editor, labellist.get_ontology().asdict())
project.datasets.connect(dataset)
project.enable_model_assisted_labeling()

True

## Add ids required for MAL

In [9]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
labellist.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .assign_schema_ids(OntologyBuilder.from_project(project)) \
         .add_to_dataset(dataset, signer)

1it [00:04,  4.47s/it]
1it [00:04,  4.81s/it]
1it [00:00, 9489.38it/s]


## MAL: Upload Predictions to a Project
* Pre-label image so that we can quickly create ground truth
* This step is not required for MEA.

In [10]:
upload_task = MALPredictionImport.create_from_objects(client, project.uid, f'mal-import-{uuid.uuid4()}', list(NDJsonConverter.serialize(labellist)))
upload_task.wait_until_done()
print(upload_task.state)

AnnotationImportState.FINISHED


In [11]:
upload_task.statuses[:5]

[{'uuid': 'b7209dc7-4633-44f8-a70a-92f5b7870d44',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': '4b3786d4-4014-470e-ab7d-de3bf1ab3db6',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': 'e4504c03-b909-4bd2-9379-8aef20bd42e9',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': 'a99a32e4-a0de-431d-b934-17d72d8221db',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': '14e965e5-eaf5-4ef3-a0c8-af7d77052da1',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'}]

## Label
* Create ground truth data for MEA
* Click on link below to label

In [12]:
print(f"https://app.labelbox.com/projects/{project.uid}")

https://app.labelbox.com/projects/ckrmjytxb02gz0y7407itatfe


In [13]:
labels = project.export_labels().as_list()

## MEA
1. Create a model
    * Think of this as a model that you want to perform experiments on
2. Create a model run
    * Think of this as a single experiment for a particular model.
    * E.g. this model run is for an instance of a model with particular hyperparameters
3. Select the ground truth annotations for analysis
4. Compute metrics
4. Upload model predictions and metrics to labelbox

In [15]:
model = client.create_model(name = "test-model-10", ontology_id = project.ontology().uid)
model_run = model.create_model_run('test-run-1')

In [16]:
model_run.upsert_labels([label.uid for label in labels])

True

In [17]:
label_lookup = {label.data.uid : label for label in labels}

In [43]:
for pred in labellist:
    label = label_lookup[pred.data.uid]
    score = data_row_miou(label, pred)
    if score is None:
        continue
        
    pred.annotations.append(
        ScalarMetric(value = score)
    )
    

0.9898928623589912


In [46]:
upload_task = model_run.add_predictions(f'mea-import-{uuid.uuid4()}', list(NDJsonConverter.serialize(labellist)))

In [47]:
upload_task.wait_until_done()
print(upload_task.state)

AnnotationImportState.FINISHED


In [48]:
upload_task.statuses

[{'uuid': '1358893f-71a3-4a52-9530-43e70344fa53',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': '5ec325fa-f6ef-4ee4-81d2-33d87208700c',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': '31b07ea2-6eae-4a20-9512-c8675df1a07e',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': '8ac56363-43c2-4741-8b3b-c1874db7b595',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': '419b8d07-566b-4f80-9a5c-60f09f4e3b76',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': 'f273862a-fb6f-4c9b-8cb3-b0ef282f4ab4',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': '0fe7e725-0f4c-4013-8f2a-2c1b175f2c85',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid': 'ae4f6cf2-d8f3-47c1-bbb6-ab3c16c152ac',
  'dataRow': {'id': 'ckrmjz5og00620yu03vjrbpxd'},
  'status': 'SUCCESS'},
 {'uuid'

In [49]:
# View the uploaded predictions and labels (annotation group) in the web app
annotation_group = next(model_run.annotation_groups())
print(annotation_group.url)

https://app.labelbox.com/models/9be7b30f-7142-07a1-663f-aa082cdb592f/9be7b30f-d74c-0cff-9225-1b471f965c8a/AllDatarowsSlice/21f1fa97-d81f-4623-bfce-7bfc81999a84?view=carousel
